# Human Activity Recognition
---

In [1]:
import numpy as np
import pandas as pd

In [2]:
human = pd.read_csv("./HumanActivityRecognition/train.csv")
human.shape

(7352, 563)

## 데이터 준비
- 결측치
- 중복치
- x, y 분리
- y값 인코딩
- 정규화 (표준화)
- GridSearchCV 베스트 모델, 파라미터 찾기
- 베스트 모델로 test 파일 읽어서 score 계산

In [4]:
human.head(3)

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING


In [17]:
human.isnull().sum()

tBodyAcc-mean()-X       0
tBodyAcc-mean()-Y       0
tBodyAcc-mean()-Z       0
tBodyAcc-std()-X        0
tBodyAcc-std()-Y        0
                       ..
angle(X,gravityMean)    0
angle(Y,gravityMean)    0
angle(Z,gravityMean)    0
subject                 0
Activity                0
Length: 563, dtype: int64

In [18]:
human.duplicated().sum()

0

In [24]:
human['Activity'].unique()

array(['STANDING', 'SITTING', 'LAYING', 'WALKING', 'WALKING_DOWNSTAIRS',
       'WALKING_UPSTAIRS'], dtype=object)

In [25]:
df = human.copy()
df['Activity'] = df['Activity'].map({ 'STANDING':0,
                                    'SITTING':1,
                                    'LAYING':2,
                                    'WALKING':3,
                                    'WALKING_DOWNSTAIRS':4,
                                    'WALKING_UPSTAIRS':5})
df.head(3)

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,0
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,0
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,0


In [27]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

## 표준화

In [29]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)

X_train = scaler.transform(X)
y_train = y.values

X_train.shape, y_train.shape

((7352, 562), (7352,))

## GridSearchCV 

In [93]:
from sklearn.svm import SVC

param_range = [0.1, 0.15, 0.2]

params = [
    {   # key는 parameter 이름과 같아야 함
        'C' : param_range,
        'gamma' : param_range,
        'kernel' : ['rbf']
    },
    {
        'C' : param_range,
        'kernel' : ['linear']
    },
    {
        'C' : param_range,
        'degree' : [2,3],
        'kernel' : ['poly']
    }
]

In [94]:
from sklearn.model_selection import GridSearchCV

clf = SVC(random_state=2022)
gs = GridSearchCV(estimator=clf,
                    param_grid=params,
                    scoring ='accuracy', 
                    cv=3,
                    n_jobs=-1,
                    verbose=3
                 )
gs.fit(X_train, y_train)

Fitting 3 folds for each of 28 candidates, totalling 84 fits


GridSearchCV(cv=3, estimator=SVC(random_state=2022), n_jobs=-1,
             param_grid=[{'C': [0.095, 0.1, 0.15, 0.2],
                          'gamma': [0.095, 0.1, 0.15, 0.2], 'kernel': ['rbf']},
                         {'C': [0.095, 0.1, 0.15, 0.2], 'kernel': ['linear']},
                         {'C': [0.095, 0.1, 0.15, 0.2], 'degree': [2, 3],
                          'kernel': ['poly']}],
             scoring='accuracy', verbose=3)

In [95]:
gs.best_estimator_

SVC(C=0.2, kernel='linear', random_state=2022)

In [96]:
gs.best_score_

0.9383854986302967

In [97]:
gs.best_params_

{'C': 0.2, 'kernel': 'linear'}

In [105]:
clf = SVC(C=0.1, kernel='linear', random_state=2022)
#clf = SVC(C=0.2, kernel='linear', random_state=2022)
clf.fit(X_train, y_train)

SVC(C=0.1, kernel='linear', random_state=2022)

In [106]:
clf.score(X_train, y_train)

0.9934711643090316

## Test score

In [107]:
df2 = pd.read_csv("./HumanActivityRecognition/test.csv")
df2.head(3)

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.257178,-0.023285,-0.014654,-0.938404,-0.920091,-0.667683,-0.952501,-0.925249,-0.674302,-0.894088,...,-0.705974,0.006462,0.162920,-0.825886,0.271151,-0.720009,0.276801,-0.057978,2,STANDING
1,0.286027,-0.013163,-0.119083,-0.975415,-0.967458,-0.944958,-0.986799,-0.968401,-0.945823,-0.894088,...,-0.594944,-0.083495,0.017500,-0.434375,0.920593,-0.698091,0.281343,-0.083898,2,STANDING
2,0.275485,-0.026050,-0.118152,-0.993819,-0.969926,-0.962748,-0.994403,-0.970735,-0.963483,-0.939260,...,-0.640736,-0.034956,0.202302,0.064103,0.145068,-0.702771,0.280083,-0.079346,2,STANDING


In [108]:
df2['Activity'] = df2['Activity'].map({ 'STANDING':0,
                                    'SITTING':1,
                                    'LAYING':2,
                                    'WALKING':3,
                                    'WALKING_DOWNSTAIRS':4,
                                    'WALKING_UPSTAIRS':5})

In [109]:
X_test = df2.iloc[:,:-1]
y_test = df2.iloc[:,-1]

In [110]:
X_test = scaler.transform(X_test)
y_test = y_test.values

X_test.shape, y_test.shape

((2947, 562), (2947,))

In [111]:
clf.score(X_test, y_test)

0.9606379368849678